# ARP Lab
## A Retirement Planning Laboratory

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. This does not mean one cannot make decisions. These decisions need to be guided with an understanding of the sensitivity of the parameters.This is exactly where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand your financial situation.

Copyright - Martin-D. Lacasse (2023)

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no guarantee. Use at your own risk.*

# Exploring the complex effects of Roth conversions

In this short notebook, we will explore the subtle effects of Roth conversions.

A common argument made about Roth conversions is that their benefits only depend on the tax rate now vs. the tax rate in the future. Mathematically, we have an amount of savings $A$ in a tax-deferred account yielding an average annual return rate $r$ over $n$ years. The nominal value of our savings at year $n$ will be denoted $A_n$, and can be expressed as follows,

$ A_n = A (1 + r)^n. $

If we withdraw $A_n$ after $n$ years, it will be subject to a tax rate $t_n$. Let's call this net amount $B_n$ obtained after withdrawing the money and paying income tax on it,

$B_n = A_n (1 - t_n) = A (1 + r)^n (1 - t_n).$

If on the other hand, we convert the money today at a tax rate \$t_0\$, we would invest $A (1 - t_0)$ in a tax-free account that would grow at the same rate $r$ during $n$ years. The resulting amount $B'_n$ would then be:

$B'_n = A (1-t_0) (1 + r)^n = A (1+r)^n (1-t_0).$

Comparing the two last equations, it should be clear that if $t_0 = t_n$, then $B_n = B'_n$ due to the commutative property of multiplications.

The common conclusion drawn from this argument is that unless the tax rate today is smaller than the tax rate in the future, i.e., unless $t_0 < t_n$, which would imply that $B'_n > B_n$, then there is no benefit in performing a Roth conversion. It is often argued that the income at retirement will be lower than during active life, and therefore the tax rate will be lower in retirement, provided the tax code stays as it is. However, speculations about future tax rates, some of which having some merit, are then brought up. For example, it is observed that the historical income tax rates in the US are not sustainable given the mandatory spending of the US government, especially regarding an aging population. To support this point, historical data on income tax rates are shown to support the hypothesis that tax rates are likely to increase in the future. The following figure shows how marginal tax rates have decreased during the last 50 years, as a supporting evidence that an increase of income tax are likely in the future.
![US historical income tax rate](images/USIncomeTaxMarginalRates.png)

So what is wrong with this argument? Nothing. The argument is valid, the assumptions are not. Here are the hidden assumptions:
- The rate of return $r$ is constant in time. **It is obviously not.**
- The income at retirement is likely to be less than during one's active life, and therefore income taxes during retirement will be lower. **This is not the case if the required minimum distributions (RMDs) are large.** This is particularly true for individuals having large tax-deferred accounts.
- The rate of return in a tax-deferred account is the same as the one in a tax-free account. **Not necessarily, in fact they shouldn't be.** For tax and risk management, the distribution of assets in these accounts should definitely not be the same as both withdrawals and investments from and in these accounts should be coordinated. An easy strategy is to keep the high-risk/high-reward assets in the tax-free account, which is likely to be depleted last, and put the less volatile assets in the tax-deferred account. This means that $r$ for the tax-deferred account will definitely not be the same as the one for the tax-free account.

ARP Lab is not free of assumptions. Future tax rates are known until 2025. For after 2025, we assume that the tax code of 2017 becomes active after the expiration of the TCJA (expiring in 2025). Income brackets are indexed for inflation, which rate is speculated through the rates selected (e.g., historical, fixed, or statistical) by the user.

Let's investigate this in more detail by using a few specific examples. We will be using historical data and a heuristic Roth optimizer. Note that this optimizer is not trying to get the maximum income during retirement. Instead, it tries to maximize the estate under the constraint that the desired net income will be matched. Because Clyde and Bonnie have sufficient assets, they believe that they can live comfortably while leaving some money to their heirs. The goal is therefore to maximize the estate, under the assumption that a certain income tax rate will apply when heirs withdraw the money from the tax-deferred account. This rate is not an estate tax, it is an income tax rate on the tax-deferred portion of the estate.

### Loading the modules - housekeeping
Let's start with some housekeeping. The following commands load the required module and set the program to display informational messages as it runs. It also makes sure that all graphs are properly displayed within the jupyter interface.

In [ ]:
%matplotlib inline
import arp

arp.setVerbose(True)

## Initialize

We start with a couple having a little over \\$2M in assets. As is common, one spouse can have significantly more assets than the other. Here, Bonnie enjoyed high wages and generous benefits from her employer, while Clyde had an equally enjoyable but less lucrative career. Let's assume that Clyde and Bonnie are about 5 years from retirement. Bonnie is born in 64, while Clyde is two years younger. Their life expectancy is assumed to be a few years longer than average for their age.

In [ ]:
plan = arp.Plan(YOB=[1964, 1966], expectancy=[87, 84])

## Assets balances in each type of savings accounts
You should be familiar with this command by now. Clyde has \\$100 k in her taxable savings account, \\$2 M in her 401k, and \\$120 k in a Roth 401k account. Clyde has \\$50 k in is taxable savings account, \\$400 k in a 403b, and \\$25 k in a Roth IRA. They leave everything to each others, indicated by the full fraction 1.  as beneficiaries.

In [ ]:
plan.setAssetBalances(taxable=[100000, 50000], 
                       taxDeferred=[2000000, 400000],
                       taxFree=[120000, 25000],
                       beneficiary=[1, 1])

## Contributions, earnings, and Roth conversions

This file contains anticipated income and contributions to savings accounts over the next few years. No Roth conversions are currenly planned in the file. Have a look at the worksheet in Excel. Very simple.

In [ ]:
plan.readContributions('roth.xlsx')

##  Assets allocation ratios for each type of savings account

We will use a fully coordinated assets allocation strategy. This means that the overall assets allocation is maintained for the sum of both spouses, but the ratio of each can be biased depending on the balance in savings accounts. This way, the spouse with a larger balance in a tax-free account can carry a larger fraction of stock equities, while still maintaining the requested assets allocation when considering the sum of the accounts of both spouses. Being conservative, they start with a 60/40 portfolio, gliding to a 70/30 portfolio at the midst of their retirement life.

In [ ]:
# For coordination taking place for each individual separately.
# plan.setCoordinatedAR(initial=[[60, 30, 5, 5], [70, 30, 0, 0]], final=[[50, 30, 5, 15], [40, 20, 15, 25]])

# For a single individual, of for coordinating both spousal accounts simultaneously.
plan.setCoordinatedAR(initial=[60, 30, 5, 5], final=[70, 20, 5, 5])

# An interpolation must follow either call for generating the data for all the years of the simulation.
# This determines how the glide takes place.
plan.interpolateAR('s-curve')
plan.showAllocations()

## Desired income and income profile

Our couple believes that \\$120k net income can be sufficient for their needs. As they want to spend more at the beginning of their retirement, they opt for a *smile* spending profile.

In [ ]:
plan.setDesiredIncome(120000, 'smile')

## Pension and social security
They have no pension, but both will receive a little over average for social security benefits (average social security benefits are about \\$22k for men and \\$18k for women). Clyde will take social security benefits at full retirement age, while Clyde will wait until age 70.

In [ ]:
plan.setPension([0, 0], [65, 65])
plan.setSocialSecurity([35000, 27000], [70, 67])

## Spousal split
This determines how to withdraw from each spousal account. The option *auto* makes it proportional to the balance, avoiding overdrawing from an account.

In [ ]:
plan.setSpousalSplit('auto')

## Set rates and run case

We want to find the (near) best Roth conversions that can maximize the wealth at the end of life. Note that this is not an exercise trying to minimize the amount of tax paid or to maximize the amount to spend. More taxes could be paid, if this leads to more wealth for the heirs conditional to satisfying a desired net income during the life span of Clyde and Bonnie.

In particular, is there a preferable sequence of conversions when markets are fluctuating? To answer this question, we will use two cases. One with fixed rate of returns, and another with the sequence of historical rates that happened from 1969 and the years after. Are there signs in the market that can indicate preferable times to make a Roth conversion?

We start by making clones of the plan we have designed so far, and select different kind of rates. One has fixed, conservative rates, while the other has historical and realistic annual rates of returns.

In [ ]:
plan_fixed = arp.clone(plan)
plan_fixed.setRates('conservative')

plan_hist = arp.clone(plan)           
plan_hist.setRates('historical', 1969)

Let's first run both cases with no conversion. When reporting estate value, we will assume that there marginal income tax rate to be paid by heirs on the tax-deferred portion of the inheritance is 20%.  We will use the variable `heirsTaxRate` to store the 20%, allowing you to change the value at a single place for experimenting with different scenarios.

Note that the estate value is in today's \\$.

In [ ]:
arp.setVerbose(False)
heirsTaxRate = 20

plan_fixed.run()
plan_fixed.estate(heirsTaxRate)

plan_hist.run()
plan_hist.estate(heirsTaxRate)

Given the historical sequence of rates, including inflation, the final wealth in today's \\$ is about \\$5.1M, while it is only \\$3.2M when we consider fixed conservative rates. This is to be expected.

Now we try to find a better scenario that would include the (near) best sequence of Roth conversions for each of these cases. For this purpose, we use the `optimizeRoth()` function. The first and second arguments specify the plan to optimize and the tax rate on the tax-deferred portion of the estate. The minimization algorithm must also be given a Roth conversion amount to start considering down to the smallest adjustment to make. These values are specified using the `startConv` and `minConv` values. Note that the computer time required to make the calculations will increase as you decrease the value of `minConv`.

We will use the `%%time` option of Jupyter to report the computer time to run the case. We will assume a tax rate of 20% to be applied to the tax-deferred portion left in the estate after Clyde and Bonnie have passed. This is done to be able to make meaningful comparisons between end-of-life in the case where portfolios have a mix of tax-deferred and tax-free assets. 

In [ ]:
%%time
plan_fixed2, conversions_fixed = arp.optimizeRoth(plan_fixed, heirsTaxRate, minConv=250, startConv=128000)
print(conversions_fixed)

For fixed rates, we see that Roth conversion can still increase the value of the estate by \\$186 k. This is mostly due to tax avoidance of the large RMDs that would result otherwise. 

In [ ]:
plan_fixed2.estate(heirsTaxRate)

Let's compare the fixed-rate case with and without Roth conversions. Notice how the composition of the savings accounts at the end of life is quite different between the two cases. We use identifying strings in the function calls for plots to distinguish the graphs from one another. Also notice how large are Bonnie's RMDs when no conversions are performed and that Roth conversions are made during the years when the income of the couple is lower, i.e., between the time when wages stopped coming in and social security starts.

In [ ]:
plan_fixed.showSources('fixed')
plan_fixed2.showSources('fixed+conversions')

plan_fixed.showRates('fixed')

plan_fixed.showAccounts('fixed')
plan_fixed2.showAccounts('fixed+conversions')

plan_fixed.showGrossIncome('fixed')
plan_fixed2.showGrossIncome('fixed+conversions')

plan_fixed.showTaxes('fixed')
plan_fixed2.showTaxes('fixed+conversions')

Now let's optimize the case with historical rates.

In [ ]:
%%time
plan_hist2, conversions_hist = arp.optimizeRoth(plan_hist, heirsTaxRate, minConv=250, startConv=128000)
print(conversions_hist)

We look at the estate value of the historical rates with Roth conversions.

In [ ]:
plan_hist2.estate(heirsTaxRate)

Even when assuming a rather low income tax rate of 20% on the tax-deferred account for the heirs, Roth conversions could increase the value by close to \\$775k (in today's \\$, or strictly speaking in that case, in 1969 \\$, which is the first year of retirement).

We can save all withdrawals and events using the `saveInstance()` function. Do `help(arp.Plan.saveInstance)` for details.

We do the same comparison, without and with Roth conversions with the plan having historical rates from 1969. Pay special attention to the alignments between conversions and market recoveries.  In particular, compare the suggested conversions with the fluctuations in the market.

In [ ]:
plan_hist.showSources('hist')
plan_hist2.showSources('hist+conversions')

plan_hist.showRates('hist')

plan_hist.showAccounts('hist')
plan_hist2.showAccounts('hist+conversions')

plan_hist.showGrossIncome('hist')
plan_hist2.showGrossIncome('hist+conversions')

plan_hist.showTaxes('hist')
plan_hist2.showTaxes('hist+conversions')

### Brief analysis
Note how the Roth conversions are timed with the spikes in rates of return, reinforcing the practice that making Roth conversion during the downturns of the market is an excellent strategy. Here, the Roth conversions are assumed to be done early in the year, before the annual rate of return is applied. As a results, conversions in the same year as when the market recovers is the best strategy. As we do not know when the market will bounce, we do know when the market goes down, and therefore a strategy of converting any time during a market large dip is good. Maybe a guardrail approach could be investigated.

Another contributing factor to this additional wealth is that when tax-deferred assets are large, the RMDs can have a significant tax impact in the middle-to-late years of retirement. Roth conversion smooths out these income peaks by shifting the tax burden earlier in the retirement and avoiding large RMDs later in life. Moreover, coordinated accounts can maintain a balanced portfolio while shifting high-performance (but riskier) assets to a tax-free account, further reducing the tax burden on the remaining assets.

When return rates are constant, the proposed Roth conversions will tend to align with a given tax bracket (dashed lines), following common wisdom. The situation here is a combination of both, as it seems advantageous to break the tax bracket line for generating more return in a tax-free account before a swing of market recovery.

Recall that this simulation is using average annual return rates and not daily rates. This has two consequences. The first one is that a knowledge of daily market values could lead to even more gains. But this is only wishful thinking. Second, this simulation assumes that conversions happen at the beginning of the year. A practical implementation would be to design a guard rail system in which a drop in the market beyond a certain percentage value could trigger a Roth conversion of a certain amount optimized over informed assumptions.

### More questions

I hope this example have convinced you that Roth conversions have complex implications and should be considered for reasons beyond those strictly comparing the current tax rate with possible tax rates of the future. The effects of large RMDs, heirs income tax on estate, and market fluctuations need to be considered.

Many more questions can be explored:
- What is the effect of the tax-deferred tax rate assumed? Does a rate of 30% give the same answer as 20%? How about 60%? At what percentage will optimized scenarios have the tax-deferred accounts all be converted by the time that Clyde and Bonnie have passed?
- What is the effect of a *smile* spending profile vs a *flat* spending profile?
- What if both spouses have large tax-deferred assets?
- What are the benefits of accounts allocations in this case? What if accounts allocations are not coordinated?
- What happens if a different historical starting year other than 1969 is selected? Are the effects as significant?
- ...

ARP Lab is designed to tackle these questions. They can be answered by changing only a few parameters in this file. Or in your own case file. For running a new case, edit the parameters and click on the double play arrows at the top bar of the notebook (*Restart Kernel and Run All Cells*).

Go explore!
  